# EEG - Flow

## 2. Select bad channels and annotate bad segments

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

Last edit: 06.05.2023 01:46
@anguyen

In [ ]:
import os

from mne.io import write_info
from mne.viz import set_browser_backend

%matplotlib qt
set_browser_backend('qt')

from eeg_flow.tasks.prep_annotations import(
    load_for_annotations,
    check_bridges,
    interpolate_bridge_try,
    auto_bad_channels,
)

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
"""
PARTICIPANT: str (for example "P03", "P28")
GROUP:       str ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8"]
TASK:        str, ["oddball", "UT"]
RUN:         int [1,2]
"""

PARTICIPANT = "P02"
GROUP       = "G2"
TASK        = "oddball"
RUN         = 1

In [ ]:
DERIVATIVES_SUBFOLDER, FNAME_STEM, raw, locks = load_for_annotations(PARTICIPANT, GROUP, TASK, RUN)

## 2.1 Bridges

More information can be found on this [MNE tutorial](https://mne.tools/stable/auto_examples/preprocessing/eeg_bridging.html).

In [ ]:
check_bridges(DERIVATIVES_SUBFOLDER, FNAME_STEM, raw)

If the bridges don't look fixable, the recording should be probably dropped. Else, they are interpolated:

In [ ]:
interpolate_bridge_try(raw)

## 2.2 Bad channels

Bad channels are suggested by `pyprep`. A visual inspection is however needed to confirm or modify the output from `pyprep`.

In [ ]:
%%time
auto_bad_channels(raw)

## 2.3 Visual inspection and annotations of bad segments

During the visual inspection, bad segments should be annotated.

In [ ]:
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)
raw.plot(theme="light")

In [ ]:
annotations = raw.annotations

## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
FNAME_INFO = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_info.fif")
assert not FNAME_INFO.exists()  # write_info always overwrites
write_info(FNAME_INFO, raw.info)
FNAME_ANNOT = (
    DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step2_oddball_with_bads_annot.fif")
)
annotations.save(FNAME_ANNOT, overwrite=False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway